## Example workflow to register multi-view light sheet data

This notebook uses the [multiview example dataset](https://drive.google.com/file/d/1VFT2APVPItBCyzrQ7dPWBNILyNh6yDKB/view?usp=sharing) provided in the [BigStitcher-Spark github repo](https://github.com/JaneliaSciComp/BigStitcher-Spark).

Notes:
- registration: this notebook uses translation and affine registration
- fusion: currently linear blending and content-based fusion are supported
- visualization:
  - neuroglancer (no installation required)
  - (optional) install [napari-stitcher](https://multiview-stitcher.github.io/napari-stitcher/main/)

In [1]:
import os
import numpy as np
from pathlib import Path
from tqdm import tqdm
import dask.diagnostics, tempfile

from multiview_stitcher import msi_utils, spatial_image_utils

%matplotlib notebook

# the package napari-stitcher is required for visualization with napari
VISUALIZE_USING_NAPARI = False

if VISUALIZE_USING_NAPARI:
    import napari
    from napari_stitcher import viewer_utils

## Specify input data

In [2]:
# Download the example dataset
# https://drive.google.com/file/d/1VFT2APVPItBCyzrQ7dPWBNILyNh6yDKB/view?usp=sharing
# and indicate the extracted paths below

base_dir = Path('../image-datasets/bigstitcher/IP_TIFF')
filenames = list(base_dir.glob('*TL18*.tif'))

# extract angle from e.g. spim_TL18_Angle135.tif
angles = [int(f.stem.split('Angle')[1].split('.')[0]) for f in filenames]

# sort filenames by angle
filenames = [filenames[i] for i in np.argsort(angles)]
angles = np.sort(angles)

if not len(filenames):
    raise ValueError('No files found. Please download the example dataset and set the correct path.')

print('Files:', [f.name for f in filenames])
print('Extracted angles:', angles)


Files: ['spim_TL18_Angle0.tif', 'spim_TL18_Angle45.tif', 'spim_TL18_Angle90.tif', 'spim_TL18_Angle135.tif', 'spim_TL18_Angle180.tif', 'spim_TL18_Angle225.tif', 'spim_TL18_Angle270.tif']
Extracted angles: [  0  45  90 135 180 225 270]


## Convert input data to OME-Zarr

First the convert the input views to OME-Zarr format. We indicate the scale stored in the imagej metadata (TODO: confirm).

In [3]:
from multiview_stitcher import io, ngff_utils
from multiview_stitcher import spatial_image_utils as si_utils

# read input tiff files into spatial images
sims = [
    io.read_tiff_into_spatial_xarray(
        filename,
        dims=('z', 'y', 'x'),
        scale={'z': 2., 'y': 0.7310781, 'x': 0.7310781})
    for filename in filenames
]

# write tiles to ome zarr format
for sim, filename in zip(sims, filenames):
    sim = ngff_utils.write_sim_to_ome_zarr(
        sim,
        filename.with_suffix('.ome.zarr'),
        overwrite=True
    )

/Users/albertm/.pixi/envs/conda/envs/ms/lib/python3.11/site-packages/zarr/api/asynchronous.py:1257: ZarrRuntimeWarning: The `write_empty_chunks` keyword argument is deprecated and will be removed in future versions. To control whether empty chunks are written to storage, either use the `config` keyword argument, as in `config={'write_empty_chunks': True}`,or change the global 'array.write_empty_chunks' configuration variable.
  _warn_write_empty_chunks_kwarg()
/Users/albertm/.pixi/envs/conda/envs/ms/lib/python3.11/site-packages/zarr/api/asynchronous.py:1036: ZarrRuntimeWarning: The `write_empty_chunks` keyword argument is deprecated and will be removed in future versions. To control whether empty chunks are written to storage, either use the `config` keyword argument, as in `config={'write_empty_chunks': True}`,or change the global 'array.write_empty_chunks' configuration variable.
  _warn_write_empty_chunks_kwarg()
/Users/albertm/.pixi/envs/conda/envs/ms/lib/python3.11/site-packages/z

## Load the input data from OME-Zarr

`ngff_utils.read_sim_from_ome_zarr` will only reference the data on disk, so the data will not be loaded into memory.

### Registration

For registration, we load the spatial images at resolution level 1 (one lower than 0).

In [4]:
# load spatial images for registration
sims_reg = [
    ngff_utils.read_sim_from_ome_zarr(
        filename.with_suffix('.ome.zarr'),
        resolution_level=1,
    ) for filename in filenames]

# the next two lines can be executed if registration should only be done on a
# subset of the views in which only the beads are visible
# otherwise if commented out, the registration will be done on the full field of view
for iview, sim in enumerate(sims_reg):
    sims_reg[iview] = si_utils.sim_sel_coords(sims_reg[iview], {'x': slice(800, 1100)})

# convert spatial images to multiscale spatial images which the registration step expects
msims_reg = [msi_utils.get_msim_from_sim(
    sim, scale_factors=None)
    for sim in sims_reg]

### Fusion

For fusion, we load the full resolution data at resolution level 0. This is the data that will be used for the final fusion step.

In [5]:
sims_fus = [
    ngff_utils.read_sim_from_ome_zarr(
        filename.with_suffix('.ome.zarr'),
        resolution_level=0,
    ) for filename in filenames]

msims_fus = [msi_utils.get_msim_from_sim(
    sim, scale_factors=None)
    for sim in sims_fus]

## Set estimate of initial transformations

We set the initial transformations estimated from the angles indicated in the filenames. We're assuming they represent a rotation around the center of the views.

These transformations will be refined during the registration process.

In [6]:
from multiview_stitcher import param_utils

for iview in range(len(angles)):

    # define rotation transformation around x axis
    affine = param_utils.affine_from_rotation(
        angles[iview] / 180 * np.pi,
        point=spatial_image_utils.get_center_of_sim(msims_reg[iview]['scale0/image'], transform_key=None),
        direction=[0,0,1],
        )
    
    # convert matrix to xarray.DataArray with labeled axes
    xaffine = param_utils.affine_to_xaffine(affine)

    # set the transformations on the image objects used for registration and fusion
    msi_utils.set_affine_transform(
        msims_reg[iview],
        affine[None], # one tp
        transform_key='affine_manual',
    )


### Visualize pre-registered views

In [7]:
if VISUALIZE_USING_NAPARI:

    viewer = napari.Viewer(ndisplay=3)
    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims_reg, transform_key='affine_manual', n_colors=4, contrast_limits=[0, 10])
    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)
    viewer.reset_view()

### Register views

First we apply a translation registration on the (pre-rotated) views to perform a first coarse alignment.

In [8]:
from multiview_stitcher import registration

with dask.diagnostics.ProgressBar():

    # phase shift registration
    params = registration.register(
        msims_reg,
        # registration_binning={'z': 2, 'y': 8, 'x': 8},
        registration_binning={'z': 1, 'y': 4, 'x': 4},
        reg_channel_index=0,
        transform_key='affine_manual',
        new_transform_key='translation_registered',
        pre_registration_pruning_method=None,
        groupwise_resolution_kwargs={
            'transform': 'translation',
        },
        n_parallel_pairwise_regs=None, # limit this to e.g. 2 to reduce memory usage
    )

    # # alternatively registration using ANTsPy
    # params = registration.register(
    #     msims_reg,
    #     registration_binning={'z': 2, 'y': 8, 'x': 8},
    #     reg_channel_index=0,
    #     transform_key='affine_manual',
    #     new_transform_key='translation_registered',
    #     pre_registration_pruning_method=None,
    #     pairwise_reg_func=registration.registration_ANTsPy,
    #     pairwise_reg_func_kwargs={
    #         'transform_types': ['Translation'],
    #     },
    #     groupwise_resolution_kwargs={
    #         'transform': 'translation',
    #     }
    # )
    

[########################################] | 100% Completed | 1.35 ss
[########################################] | 100% Completed | 413.69 ms
[########################################] | 100% Completed | 318.46 ms
[########################################] | 100% Completed | 526.64 ms
[########################################] | 100% Completed | 622.43 ms
[########################################] | 100% Completed | 623.86 ms
[########################################] | 100% Completed | 210.23 ms
[########################################] | 100% Completed | 625.36 ms
[########################################] | 100% Completed | 313.82 ms
[########################################] | 100% Completed | 628.17 ms
[########################################] | 100% Completed | 727.59 ms
[########################################] | 100% Completed | 626.78 ms
[########################################] | 100% Completed | 736.30 ms
[########################################] | 100% Completed | 313.

### Visualize registration (translation)

In [9]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=3)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims_reg,
        transform_key='affine_registered',
        n_colors=100,
        name_prefix='registered view',
        contrast_limits=[0, 10],
        positional_cmaps=True,
        )
    
    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=False)

## Affine registration

In this step we refine the transformations using an affine registration.

In [10]:
input_transform_key = 'translation_registered'
output_transform_key = 'affine_registered'

# Number of iterations for the registration
N_iter = 1
for iteration in range(N_iter):

    if iteration == N_iter - 1:
        iter_output_transform_key = output_transform_key
    else:
        iter_output_transform_key = f'{output_transform_key}_iter{iteration}'

    # rigid registration
    params = registration.register(
        msims_reg,
        # registration_binning={'z': 2, 'y': 8, 'x': 8},
        registration_binning={'z': 1, 'y': 4, 'x': 4},
        reg_channel_index=0,
        transform_key=input_transform_key,
        new_transform_key=iter_output_transform_key,
        pre_registration_pruning_method=None,
        pairwise_reg_func=registration.registration_ANTsPy,
        pairwise_reg_func_kwargs={
            # 'transform_types': ['Rigid'],
            'transform_types': ['Rigid', 'Affine'],
        },
        groupwise_resolution_kwargs={
            # 'transform': 'rigid',
            'transform': 'affine',
        },
        n_parallel_pairwise_regs=None, # limit this to e.g. 2 to reduce memory usage
    )

    input_transform_key = iter_output_transform_key

### Visualize registration

In [11]:
if VISUALIZE_USING_NAPARI:
    
    viewer = napari.Viewer(ndisplay=3)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims_reg,
        transform_key='affine_registered',
        n_colors=100,
        name_prefix='registered view',
        contrast_limits=[0, 10],
        positional_cmaps=True,
        )
    
    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds, do_link_layers=False)

## Set the obtained transformations on the images to be fused

In [12]:
fusion_transform_key = 'affine_registered'

for iview, msim in enumerate(msims_reg):
    p = msi_utils.get_transform_from_msim(msim, fusion_transform_key).sel(t=0)
    si_utils.set_sim_affine(sims_fus[iview], p, fusion_transform_key)

## Fuse views (linear blending)

In [13]:
from multiview_stitcher import fusion, weights

# define the fused output
# 'fused' is a dask array
fused = fusion.fuse(
    sims_fus[:],
    transform_key=fusion_transform_key,
    output_spacing={dim: 5 for dim in ['z', 'y', 'x']},
    output_chunksize=256, # the larger the chunksize the better for very large arrays
    # weights_func=weights.content_based, # uncomment for content-based fusion
    # weights_func_kwargs={"sigma_1": 5, "sigma_2": 11} # uncomment for content-based fusion
    )

print('Summary of the fused dask array:')
fused

Summary of the fused dask array:


<xarray.DataArray 'image' (t: 1, c: 1, z: 157, y: 182, x: 207)> Size: 6MB
dask.array<getitem, shape=(1, 1, 157, 182, 207), dtype=uint8, chunksize=(1, 1, 157, 182, 207), chunktype=numpy.ndarray>
Coordinates:
  * t        (t) int64 8B 0
  * c        (c) <U1 4B '0'
  * z        (z) float64 1kB -245.4 -240.4 -235.4 -230.4 ... 524.6 529.6 534.6
  * y        (y) float64 1kB -29.63 -24.63 -19.63 -14.63 ... 865.4 870.4 875.4
  * x        (x) float64 2kB -1.501 3.499 8.499 ... 1.023e+03 1.028e+03
Attributes:
    transforms:  {'affine_registered': <xarray.DataArray (x_in: 4, x_out: 4)>...

## Compute the fusion

... and save the result to disk as OME-Zarr.

In [ ]:
import dask.array as da
import tempfile

output_ome_zarr_path = os.path.join(base_dir, "fused_output.zarr")

if os.path.exists(output_ome_zarr_path):
    print('Output OME-Zarr already exists:', output_ome_zarr_path)
else:
    print('Fusing views...')
    with tempfile.TemporaryDirectory() as tmpdir:

        tmp_output_file = os.path.join(tmpdir, os.path.basename(output_ome_zarr_path))

        print('Streaming fusion to temporary file:', tmpdir)
        # Write the fused data to a temporary Zarr file
        with dask.diagnostics.ProgressBar():
            fused.data = da.to_zarr(
                fused.data,
                tmp_output_file,
                overwrite=True, return_stored=True, compute=True)

            print('Creating multiscale output OME-Zarr...')
            fused = ngff_utils.write_sim_to_ome_zarr(
                fused,
                output_ome_zarr_path
            )

Fusing views...
Streaming fusion to temporary file: /var/folders/pk/0yb479fx6kxbcw2k8h8tbg7w0000gq/T/tmp05fa23pp
[########################################] | 100% Completed | 3.18 sms
Creating multiscale output OME-Zarr...
[########################################] | 100% Completed | 103.99 ms
[########################################] | 100% Completed | 106.42 ms
[########################################] | 100% Completed | 101.58 ms
[########################################] | 100% Completed | 103.46 ms
[########################################] | 100% Completed | 102.79 ms
[########################################] | 100% Completed | 103.45 ms
[########################################] | 100% Completed | 101.84 ms


### Visualize fusion in napari

If data is too large to visualize in napari, you can use neuroglancer in the cell below instead.

In [15]:
if VISUALIZE_USING_NAPARI:

    viewer = napari.Viewer(ndisplay=3)

    lds = viewer_utils.create_image_layer_tuples_from_msims(
        msims_reg, transform_key=fusion_transform_key, n_colors=4,
        name_prefix='registered view',
        contrast_limits=[0, 200]
        )

    rlayers = viewer_utils.add_image_layer_tuples_to_viewer(
        viewer, lds, do_link_layers=False)

    lds = viewer_utils.create_image_layer_tuples_from_msim(
        msi_utils.get_msim_from_sim(fused),
        transform_key=fusion_transform_key,
        name_prefix='fused',
        contrast_limits=[0, 200])

    viewer_utils.add_image_layer_tuples_to_viewer(viewer, lds)

## Visualize using neuroglancer

In [ ]:
# interrupt the notebook cell to stop the viewer

from multiview_stitcher import vis_utils

# just the fusion result
vis_utils.view_neuroglancer(
    # sims=sims_fus + [fused],
    sims=[fused],
    # ome_zarr_paths=[str(filename.with_suffix('.ome.zarr')) for filename in filenames] + [output_ome_zarr_path],
    ome_zarr_paths=[output_ome_zarr_path],
    channel_coord=0,
    transform_key="affine_registered",
)

# # or visualize the registered views and the fusion result
# vis_utils.view_neuroglancer(
#     sims=sims_fus + [fused],
#     ome_zarr_paths=[str(filename.with_suffix('.ome.zarr')) for filename in filenames] + [output_ome_zarr_path],
#     channel_coord=0,
#     transform_key="affine_registered",
# )

Opening Neuroglancer in browser...
URL: https://neuroglancer-demo.appspot.com/#!%7B%22dimensions%22%3A%7B%22t%22%3A%5B1.0%2C%22%22%5D%2C%22c%22%3A%5B1.0%2C%22%22%5D%2C%22z%22%3A%5B4.9999999999999996e-06%2C%22%22%5D%2C%22y%22%3A%5B4.9999999999999996e-06%2C%22%22%5D%2C%22x%22%3A%5B4.9999999999999996e-06%2C%22%22%5D%7D%2C%22displayDimensions%22%3A%5B%22x%22%2C%22y%22%2C%22z%22%5D%2C%22layerListPanel%22%3A%7B%22visible%22%3Atrue%7D%2C%22layout%22%3A%224panel%22%2C%22layers%22%3A%5B%7B%22type%22%3A%22image%22%2C%22source%22%3A%7B%22url%22%3A%22zarr%3A//http%3A//localhost%3A8000/fused_output.zarr%22%2C%22transform%22%3A%7B%22matrix%22%3A%5B%5B1.0%2C0.0%2C0.0%2C0.0%2C0.0%2C0.0%5D%2C%5B0.0%2C1.0%2C0.0%2C0.0%2C0.0%2C0.0%5D%2C%5B0.0%2C0.0%2C1.0%2C0.0%2C0.0%2C0.0%5D%2C%5B0.0%2C0.0%2C0.0%2C1.0%2C0.0%2C0.0%5D%2C%5B0.0%2C0.0%2C0.0%2C0.0%2C1.0%2C0.0%5D%5D%2C%22outputDimensions%22%3A%7B%22t%22%3A%5B1.0%2C%22%22%5D%2C%22c%27%22%3A%5B1.0%2C%22%22%5D%2C%22z%22%3A%5B4.9999999999999996e-06%2C%22%22%5D%2C%2

::1 - - [26/Aug/2025 19:37:54] code 404, message File not found
::1 - - [26/Aug/2025 19:37:54] "GET /fused_output.zarr/.zarray HTTP/1.1" 404 -
::1 - - [26/Aug/2025 19:37:54] code 501, message Unsupported method ('OPTIONS')
::1 - - [26/Aug/2025 19:37:54] "OPTIONS /fused_output.zarr/.zattrs HTTP/1.1" 501 -
::1 - - [26/Aug/2025 19:37:54] code 501, message Unsupported method ('OPTIONS')
::1 - - [26/Aug/2025 19:37:54] "OPTIONS /fused_output.zarr/.zarray HTTP/1.1" 501 -
::1 - - [26/Aug/2025 19:37:54] code 501, message Unsupported method ('OPTIONS')
::1 - - [26/Aug/2025 19:37:54] "OPTIONS /fused_output.zarr/zarr.json HTTP/1.1" 501 -
::1 - - [26/Aug/2025 19:37:54] code 404, message File not found
::1 - - [26/Aug/2025 19:37:54] "GET /fused_output.zarr/.zattrs HTTP/1.1" 404 -
::1 - - [26/Aug/2025 19:37:54] "GET /fused_output.zarr/zarr.json HTTP/1.1" 200 -
::1 - - [26/Aug/2025 19:37:55] "GET /fused_output.zarr/0/zarr.json HTTP/1.1" 200 -
::1 - - [26/Aug/2025 19:37:55] code 501, message Unsupporte